# Análise Exploratória dos Dados

</br>

<img src="image/python_supermercado_2025.jpg" alt="Descrição da imagem" width="80%">

O objetivo deste notebook é realizar a análise exploratória de dados (EDA - Exploratory Data Analysis) da base de dados.

<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">
<img src="/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket/image/hr-python.png" width="6.5%">

## 1. Configura as bibliotecas

In [0]:
# !pip install nfceget
# !pip install bs4
# !pip install openpyxl

In [0]:
import time
import pandas as pd
from functools import reduce
from nfceget import app
import openpyxl
import matplotlib.pyplot as plt
from utils.tic_toc import tictoc
import warnings

# Suprimir avisos do Pandas
warnings.filterwarnings('ignore')

## 2. Define os parâmetros

Aqui estão os parâmetros que deverão ser alterados conforme necessidade.

In [0]:
tic_geral = time.time()
tic = time.time()

# Diretório:
dir = '/Workspace/Users/renanscavazzini@gmail.com/github/modelos/model_supermarket'

# Caminhos para ler:
base = f'{dir}/data/notas_fiscais_supermercado.xlsx'

# Caminhos para salvar:

# Tipos dos dados:
var_tipos = {
    'chave': [
        'CHAVE',
    ],
    'data': [
        'DATA',
    ],
    'binario': [],
    'categorico': [ 
        'PERIODO',
        'CATEGORIA_PRODUTO',
    ],
    'string': [
        'SUPERMERCADO', 
        'PRODUTO',
        'UNIDADE',
    ],
    'inteiro': [
        'COD_PRODUTO', 
    ],
    'numerico': [
        'VALOR_UNIDADE', 
        'QTDE', 
        'VALOR_TOTAL',
    ],
}
var_tipos

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 2 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 3. Leitura da base de dados

In [0]:
tic = time.time()

df = pd.read_excel(base)
display(df)

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 3 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 4. Categorização

Na construção da base de dados foram realizadas duas categorizações:

1. Período do dia
2. Categoria do produto

Vamos verificar a volumetria de cada categoria dessas variáveis.

In [0]:
tic = time.time()

display(df.groupby('PERIODO')['PRODUTO'].count())

In [0]:
display(df.groupby('CATEGORIA_PRODUTO')['PRODUTO'].count())

# df[(df['CATEGORIA_PRODUTO'] == 'OUTROS') & (df['VALOR_TOTAL'] > 0)]

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 4 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 5. Tratamento de inconsistências

In [0]:
tic = time.time()

### 5.1 Tipos de dados

Garantindo que a base de dados estão do tipo correto para cada variável.

In [0]:
# Garantindo que os tipos dos dados estão corretos
for tipo, colunas in var_tipos.items():
    for coluna in colunas:
        if tipo == 'data':
            df[coluna] = pd.to_datetime(df[coluna], dayfirst=True, errors='coerce')
        elif tipo == 'binario':
            df[coluna] = df[coluna].astype('bool')
        elif tipo == 'categorico':
            df[coluna] = df[coluna].astype('category')
        elif tipo == 'string':
            df[coluna] = df[coluna].astype(str)
        elif tipo == 'inteiro':
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce').astype('Int64')
        elif tipo == 'numerico':
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')

display(df.dtypes)

In [0]:
display(df.head())

### 5.2 Produtos únicos

A base de dados contém uma chave única que representa cada produto do supermercado: o código do produto. No entanto, os nomes associados a esses produtos podem mudar ao longo do tempo. O ideal é que exista apenas um nome para cada código. Para garantir que não haja esse tipo de inconsistência, será utilizado o nome mais recente de cada produto.

Além disso, pode ser que um produto tenha códigos diferentes para o mesmo nome, pode ser que seja por exemplo um produto com sabor diferente, mas o nome não específica isso. Essa inconsistência será corrijida substrituindo o código do produto pelo código maior (mais recente), pois no final das contas o produto acaba sendo o mesmo.

In [0]:
# Códigos de produtos distintos:
print(f"Há \033[93m{df['COD_PRODUTO'].nunique()}\033[0m código(s) de produto(s) distinto(s).")

# Nomes de produtos distintos:
print(f"Há \033[93m{df['PRODUTO'].nunique()}\033[0m nome(s) de produto(s) distinto(s).")

In [0]:
# Ordenando o DataFrame por 'COD_PRODUTO' e 'DATA'
df = df.sort_values(by=['PRODUTO', 'COD_PRODUTO', 'DATA'])

# Criando um dicionário com o maior código para cada nome de produto
maior_codigo_produto = df.groupby('PRODUTO')['COD_PRODUTO'].max()

# Aplicando o maior código para cada produto
df['COD_PRODUTO'] = df['PRODUTO'].map(maior_codigo_produto)

# Criando um dicionário com o nome mais recente de cada produto
produto_mais_recente = df.drop_duplicates(subset='COD_PRODUTO', keep='last').set_index('COD_PRODUTO')['PRODUTO']

# Aplicando o nome mais recente para cada produto
df['PRODUTO'] = df['COD_PRODUTO'].map(produto_mais_recente)

print(f"Há \033[93m{df['COD_PRODUTO'].nunique()}\033[0m código(s) de produto(s) distinto(s).")
print(f"Há \033[93m{df['PRODUTO'].nunique()}\033[0m nome(s) de produto(s) distinto(s).")

In [0]:
# Nome do produto é o mesmo mas o código do produto é diferente
df_aux1 = df.groupby('COD_PRODUTO').filter(lambda x: x['PRODUTO'].nunique() > 1)
print(f"Há \033[93m{df_aux1['COD_PRODUTO'].nunique()}\033[0m código(s) de produto(s) com o mesmo código mas com nome(s) diferente(s)")
print(f"Há \033[93m{df_aux1['PRODUTO'].nunique()}\033[0m nome(s) de produto(s) com o mesmo código mas com nome(s) diferente(s)")

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 5 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 6. Propriedades

In [0]:
tic = time.time()

In [0]:
# Calcula o total de gastos
total_gastos = df['VALOR_TOTAL'].sum()

# Idas ao mercado
idas = df['CHAVE'].nunique()

# Calcula a média de gastos por nota fiscal
media_gastos = total_gastos / idas

print(f"Total de gastos: R$ {round(total_gastos,2)}")
print(f"Quantidade de idas ao mercado: {idas}")
print(f"Média de gastos por nota fiscal: {round(media_gastos,2)}")

In [0]:
# Valor total gasto por data
total_por_data = df.groupby(df['DATA'].dt.date)['VALOR_TOTAL'].sum()

# Valor total gasto por mês
total_por_mes = df.groupby(df['DATA'].dt.to_period('M'))['VALOR_TOTAL'].sum()

# Valor total gasto por ano
total_por_ano = df.groupby(df['DATA'].dt.year)['VALOR_TOTAL'].sum()

print(f"Valor total gasto por data:")
display(total_por_data)
print(f"\nValor total gasto por mês:")
display(total_por_mes)
print(f"\nValor total gasto por ano:")
display(total_por_ano)

In [0]:
# Calcula o dia que mais e menos gastou e o gasto médio por dia
dia_mais_gasto = total_por_data.idxmax()
dia_menos_gasto = total_por_data.idxmin()
media_gasto_dia = total_por_data.mean()

print(f"Dia que mais gastou: {dia_mais_gasto} com valor total de {total_por_data[dia_mais_gasto]}")
print(f"Dia que menos gastou: {dia_menos_gasto} com valor total de {total_por_data[dia_menos_gasto]}")
print(f"Gasto médio por dia: {media_gasto_dia}")

# Calcula o mês que mais e menos gastou e o gasto médio por mês
mes_mais_gasto = total_por_mes.idxmax()
mes_menos_gasto = total_por_mes.idxmin()
media_gasto_mes = total_por_mes.mean()

print(f"\nMês que mais gastou: {mes_mais_gasto} com valor total de {total_por_mes[mes_mais_gasto]}")
print(f"Mês que menos gastou: {mes_menos_gasto} com valor total de {total_por_mes[mes_menos_gasto]}")
print(f"Gasto médio por mês: {media_gasto_mes}")

# Calcula o ano que mais e menos gastou e o gasto médio por ano
ano_mais_gasto = total_por_ano.idxmax()
ano_menos_gasto = total_por_ano.idxmin()
media_gasto_ano = total_por_ano.mean()

print(f"\nAno que mais gastou: {ano_mais_gasto} com valor total de {total_por_ano[ano_mais_gasto]}")
print(f"Ano que menos gastou: {ano_menos_gasto} com valor total de {total_por_ano[ano_menos_gasto]}")
print(f"Gasto médio por ano: {media_gasto_ano}")

In [0]:
# Valor total gasto por supermercado:
total_por_supermercado = df.groupby('SUPERMERCADO')['VALOR_TOTAL'].sum()
print(f"Valor total gasto por supermercado:\n{total_por_supermercado}")

In [0]:
# Valor total gasto por período:
total_por_supermercado = df.groupby('PERIODO')['VALOR_TOTAL'].sum()
print(f"Valor total gasto por período:\n{total_por_supermercado}")

In [0]:
# Agrupa os dados por produto e soma a quantidade de cada produto
soma_quantidade = df.groupby('PRODUTO')['QTDE'].sum()

# Encontra o produto que você mais comprou
produto_mais_comprado = soma_quantidade.idxmax()

# Filtra o DataFrame para incluir apenas o produto que você mais comprou
df_produto_mais_comprado = df[df['PRODUTO'] == produto_mais_comprado]

# Certifica-se de que a coluna 'DATA' é do tipo datetime
df_produto_mais_comprado['DATA'] = pd.to_datetime(df_produto_mais_comprado['DATA'])

# Calcula a quantidade total comprada e o total gasto para o produto mais comprado
total_quantidade_produto_mais_comprado = df_produto_mais_comprado['QTDE'].sum()
total_gastos_produto_mais_comprado = df_produto_mais_comprado['VALOR_TOTAL'].sum()

print(f"O produto que mais comprei foi: {produto_mais_comprado}")
print(f"Quantidade comprada: {total_quantidade_produto_mais_comprado}")
print(f"Total gasto: {total_gastos_produto_mais_comprado}")

# Agrupa os dados por mês e soma a quantidade e o valor total
soma_por_mes = df_produto_mais_comprado.groupby(df_produto_mais_comprado['DATA'].dt.to_period('M')).agg({'QTDE': 'sum', 'VALOR_TOTAL': 'sum'})

print(f"\nSoma da quantidade e valor total do produto '{produto_mais_comprado}' por mês:\n{soma_por_mes}")

In [0]:
df_unitario = df[['PRODUTO', 'VALOR_UNIDADE', 'DATA']].drop_duplicates()
df_unitario.sort_values('DATA', inplace=True)
df_unitario.sort_values('PRODUTO', inplace=True)

# Obtém a lista de produtos únicos
produtos = df_unitario['PRODUTO'].unique()

# Para cada produto, cria uma subtabela e a ordena por data
subtabelas = {produto: df_unitario[df_unitario['PRODUTO'] == produto].sort_values('DATA') for produto in produtos}

# Agora, 'subtabelas' é um dicionário onde cada chave é um produto e cada valor é uma subtabela para aquele produto
# Você pode acessar a subtabela para um produto específico da seguinte maneira:

num_produto = 10
print(subtabelas[df_unitario['PRODUTO'].unique()[num_produto]])

In [0]:
# Cria um dicionário onde a chave é o produto e o valor é o número de linhas da subtabela
tamanho_subtabelas = {produto: len(subtabela) for produto, subtabela in subtabelas.items() if len(subtabela) > 1}

# Ordena o dicionário pelo número de linhas em ordem decrescente
tamanho_subtabelas_ordenado = dict(sorted(tamanho_subtabelas.items(), key=lambda item: item[1], reverse=True))

# Para cada produto no dicionário ordenado, imprime o índice da subtabela correspondente
for produto in tamanho_subtabelas_ordenado.keys():
    print(f"Índice da subtabela para o produto '{produto}':\n{subtabelas[produto]}\n")

In [0]:
# Para cada produto, verifica se a palavra está contida no nome do produto
palavra = 'whisky'
for produto, subtabela in subtabelas.items():
    if palavra in produto.lower():
        print(f"Índice da subtabela para o produto '{produto}':\n{subtabelas[produto]}\n")

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 6 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 7. Análise gráfica

In [0]:
tic = time.time()

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 7 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 8. Estudo de domínio

In [0]:
tic = time.time()

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 8 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 9. Análise de outliers

In [0]:
tic = time.time()

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 9 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 10. Transformações

In [0]:
tic = time.time()

In [0]:
toc = time.time()
print(f'\n\033[33mEtapa 10 | Tempo:\033[0;0m {tictoc(tic, toc)}')

## 11. Tempo decorrido

In [0]:
toc_geral = time.time()
print(f'\n\033[33mTempo decorrido no notebook:\033[0;0m {tictoc(tic_geral, toc_geral)}')

Previsão de Demanda: Use técnicas de séries temporais para prever a demanda futura de cada produto, permitindo ao supermercado ajustar seus estoques com mais eficiência e reduzir desperdícios.

Recomendação de Produtos: Desenvolva um sistema de recomendação personalizado baseado nos hábitos de compra dos clientes, sugerindo produtos que eles possam estar interessados em comprar.

Análise de Cesta de Compras: Utilize algoritmos de associação para identificar padrões de compra, como quais produtos são frequentemente comprados juntos. Isso pode ajudar a criar promoções ou combos de produtos.

Classificação de Produtos: Automatize a classificação de novos produtos com base nas descrições e características, facilitando a organização do inventário.

Análise de Sentimento: Se houver dados de feedback dos clientes, você pode aplicar técnicas de processamento de linguagem natural (NLP) para analisar o sentimento e entender melhor as opiniões dos clientes sobre os produtos.

Detecção de Anomalias: Use técnicas de detecção de anomalias para identificar transações suspeitas ou inconsistências nos dados, ajudando a detectar fraudes ou erros de entrada de dados.

Segmentação de Clientes: Aplique clustering para segmentar clientes com base em seus comportamentos de compra, permitindo campanhas de marketing mais direcionadas e eficazes.

Análise de Preços: Utilize machine learning para analisar a elasticidade de preços e identificar o impacto de mudanças de preços na demanda dos produtos.

